In [1]:
import pickle
import keras

import pandas as pd
import numpy  as np

from sklearn.preprocessing   import StandardScaler
from sklearn.metrics         import confusion_matrix
from keras.models            import Sequential
from keras.layers            import Input
from keras.layers            import Dense
from keras.layers            import BatchNormalization
from keras.layers            import Activation
from keras.layers            import Dropout
from keras.optimizers        import Adam
from keras.utils.np_utils    import to_categorical
from sklearn.model_selection import train_test_split
from tqdm                    import tqdm
from collections             import Counter

Using TensorFlow backend.


In [2]:
data_path = '../data/'

In [3]:
non_std_df = pd.read_csv(data_path + 'creditcard.csv')
non_std_df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,...,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [9]:
features    = non_std_df.drop(['Class', 'Time'], axis = 1)
target      = non_std_df['Class']
scaler      = StandardScaler()
features[:] = scaler.fit_transform(features)
features.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
count,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05
mean,-8.157366e-16,3.154853e-17,-4.409878e-15,-6.734811e-16,-2.874435e-16,4.168992e-16,-8.767997e-16,-2.423604e-16,3.078727e-16,2.026926e-17,...,2.754870e-16,1.685077e-17,1.478472e-15,-6.797197e-16,1.234659e-16,-7.659279e-16,3.247603e-16,-2.953495e-18,5.401572e-17,3.202236e-16
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,...,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-2.879855e+01,-4.403529e+01,-3.187173e+01,-4.013919e+00,-8.240810e+01,-1.963606e+01,-3.520940e+01,-6.130252e+01,-1.222802e+01,-2.258191e+01,...,-7.069146e+01,-4.741907e+01,-1.506565e+01,-7.175446e+01,-4.683638e+00,-1.975033e+01,-5.401098e+00,-5.590660e+01,-4.674612e+01,-3.532294e-01
25%,-4.698918e-01,-3.624707e-01,-5.872142e-01,-5.993788e-01,-5.010686e-01,-5.766822e-01,-4.478860e-01,-1.746805e-01,-5.853631e-01,-4.917360e-01,...,-2.746334e-01,-3.109433e-01,-7.473476e-01,-2.591784e-01,-5.854676e-01,-6.084001e-01,-6.780717e-01,-1.755053e-01,-1.604440e-01,-3.308401e-01
50%,9.245351e-03,3.965683e-02,1.186124e-01,-1.401724e-02,-3.936682e-02,-2.058046e-01,3.241723e-02,1.871982e-02,-4.681169e-02,-8.533551e-02,...,-8.104705e-02,-4.009429e-02,9.345377e-03,-1.792420e-02,6.765678e-02,3.183240e-02,-1.081217e-01,3.325174e-03,3.406368e-02,-2.652715e-01
75%,6.716939e-01,4.867202e-01,6.774569e-01,5.250082e-01,4.433465e-01,2.991625e-01,4.611107e-01,2.740785e-01,5.435305e-01,4.168842e-01,...,1.725733e-01,2.537392e-01,7.283360e-01,2.364319e-01,7.257153e-01,6.728006e-01,4.996663e-01,2.255648e-01,2.371526e-01,-4.471707e-02
max,1.253351e+00,1.335775e+01,6.187993e+00,1.191874e+01,2.521413e+01,5.502015e+01,9.747824e+01,1.675153e+01,1.419494e+01,2.180758e+01,...,5.113464e+01,3.703471e+01,1.447304e+01,3.607668e+01,7.569684e+00,1.442532e+01,7.293975e+00,7.831940e+01,1.025434e+02,1.023622e+02


In [10]:
(non_std_df['Class'] == 0).sum(), (non_std_df['Class'] == 1).sum()

(284315, 492)

In [11]:
scaler_file = open(data_path + 'scaler.pkl', 'wb')
pickle.dump(scaler, scaler_file)
scaler_file.close()

In [12]:
df = pd.concat([features, target], axis = 1)
df.to_csv(data_path + 'standardized_credit_card.csv', index = False)

In [13]:
scaler_file = open(data_path + 'scaler.pkl', 'rb')
scaler      = pickle.load(scaler_file)
df          = pd.read_csv(data_path + 'standardized_credit_card.csv')
scaler_file.close()

In [14]:
df.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000
mean,-8.155493e-16,3.228412e-17,-4.365170e-15,-6.671495e-16,-2.363387e-16,4.264686e-16,-8.693495e-16,-2.340305e-16,2.994106e-16,2.085242e-17,...,3.391983e-18,1.458816e-15,-6.835906e-16,1.216423e-16,-7.920876e-16,3.210547e-16,1.534291e-20,5.322985e-17,3.187345e-16,0.001727
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,...,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,0.041527
min,-2.879855e+01,-4.403529e+01,-3.187173e+01,-4.013919e+00,-8.240810e+01,-1.963606e+01,-3.520940e+01,-6.130252e+01,-1.222802e+01,-2.258191e+01,...,-4.741907e+01,-1.506565e+01,-7.175446e+01,-4.683638e+00,-1.975033e+01,-5.401098e+00,-5.590660e+01,-4.674612e+01,-3.532294e-01,0.000000
25%,-4.698918e-01,-3.624707e-01,-5.872142e-01,-5.993788e-01,-5.010686e-01,-5.766822e-01,-4.478860e-01,-1.746805e-01,-5.853631e-01,-4.917360e-01,...,-3.109433e-01,-7.473476e-01,-2.591784e-01,-5.854676e-01,-6.084001e-01,-6.780717e-01,-1.755053e-01,-1.604440e-01,-3.308401e-01,0.000000
50%,9.245351e-03,3.965683e-02,1.186124e-01,-1.401724e-02,-3.936682e-02,-2.058046e-01,3.241723e-02,1.871982e-02,-4.681169e-02,-8.533551e-02,...,-4.009429e-02,9.345377e-03,-1.792420e-02,6.765678e-02,3.183240e-02,-1.081217e-01,3.325174e-03,3.406368e-02,-2.652715e-01,0.000000
75%,6.716939e-01,4.867202e-01,6.774569e-01,5.250082e-01,4.433465e-01,2.991625e-01,4.611107e-01,2.740785e-01,5.435305e-01,4.168842e-01,...,2.537392e-01,7.283360e-01,2.364319e-01,7.257153e-01,6.728006e-01,4.996663e-01,2.255648e-01,2.371526e-01,-4.471707e-02,0.000000
max,1.253351e+00,1.335775e+01,6.187993e+00,1.191874e+01,2.521413e+01,5.502015e+01,9.747824e+01,1.675153e+01,1.419494e+01,2.180758e+01,...,3.703471e+01,1.447304e+01,3.607668e+01,7.569684e+00,1.442532e+01,7.293975e+00,7.831940e+01,1.025434e+02,1.023622e+02,1.000000


In [9]:
def generate_training_batch(big_class, small_class, final_proportion):
    batch_size              = int(round(big_class.shape[0] / (1 - final_proportion)))
    print('batch size', batch_size)
    small_class_sample_size = int(round(batch_size * final_proportion))  
    print('small_class_sample_size', small_class_sample_size)
    oversampled_small       = small_class.sample(small_class_sample_size, replace = True)
    
    return pd.concat([big_class, oversampled_small])

In [10]:
def df_to_numpy_arrays(df, test_prop):
    # regular transaction df
    reg_df   = df[df['Class'] == 0]
    reg_feat = reg_df.drop(['Class'], axis = 1)
    reg_targ = reg_df['Class']

    # anomalies transaction df
    anom_df   = df[df['Class'] == 1]
    anom_feat = anom_df.drop(['Class'], axis = 1)
    anom_targ = anom_df['Class']

    # splitting
    reg_feat_train, reg_feat_test, reg_targ_train, reg_targ_test     = train_test_split(reg_feat, reg_targ, test_size = test_prop)
    anom_feat_train, anom_feat_test, anom_targ_train, anom_targ_test = train_test_split(anom_feat, anom_targ, test_size = test_prop)
    
    train_feat   = pd.concat([reg_feat_train, anom_feat_train]).as_matrix()
    train_target = to_categorical(pd.concat([reg_targ_train, anom_targ_train]).as_matrix())
    test_feat    = pd.concat([reg_feat_test, anom_feat_test]).as_matrix()
    test_target  = to_categorical(pd.concat([reg_targ_test, anom_targ_test]).as_matrix())

    return train_feat, train_target, test_feat, test_target

In [11]:
def train_model(model, df, steps, epoch_start, epoch_end, fit_params, validation_prop):
    small_class_prop = df[df['Class'] == 1].shape[0] / df.shape[0]
    step_values      = np.linspace(0.5, small_class_prop, steps)
    epochs           = np.linspace(epoch_start, epoch_end, steps)
    big_class        = df[df['Class'] == 0]
    small_class      = df[df['Class'] == 1]
    
    for step, epoch in tqdm(zip(step_values, epochs)):
        training_batch                   = generate_training_batch(big_class, small_class, step)
        X_train, y_train, X_test, y_test = df_to_numpy_arrays(training_batch, validation_prop)
        fit_params['x']                  = X_train
        fit_params['y']                  = y_train
        fit_params['epochs']             = int(round(epoch))
        fit_params['validation_data']    = (X_test, y_test)
        print('Oversampling small class to %.4f%% of the training data, %d epochs' % (100 * step, fit_params['epochs']))
        model.fit(**fit_params)

In [12]:
# model = Sequential([
#     Dense(256, input_shape = (30,)),
#     BatchNormalization(),
#     Activation('relu'),
#     Dropout(0.5),
#     Dense(256),
#     BatchNormalization(),
#     Activation('relu'),
#     Dropout(0.5),
#     Dense(256),
#     BatchNormalization(),
#     Activation('relu'),
#     Dropout(0.5),
#     Dense(256),
#     BatchNormalization(),
#     Activation('relu'),
#     Dropout(0.50),
#     Dense(256),
#     BatchNormalization(),
#     Activation('relu'),
#     Dropout(0.6),
#     Dense(256),
#     BatchNormalization(),
#     Activation('relu'),
#     Dropout(0.7),
#     Dense(2, activation = 'softmax')
# ])
# model.compile(optimizer = 'adam', loss = 'binary_crossentropy')

In [13]:
model = Sequential([
    Dense(256, input_shape = (30,)),
    Activation('relu'),
    Dense(256),
    Activation('relu'),
    Dense(2, activation = 'softmax')
])
model.compile(optimizer = Adam(1e-7), loss = 'binary_crossentropy')

In [14]:
fit_params = {
    'batch_size': 2048,
    'verbose'   : 2
}

train_params = {
    'model'          : model,
    'df'             : df,
    'steps'          : 15,
    'epoch_start'    : 8,
    'epoch_end'      : 8,
    'fit_params'     : fit_params,
    'validation_prop': 0.15
}

In [15]:
train_model(**train_params)

0it [00:00, ?it/s]

batch size 568630
small_class_sample_size 284315
Oversampling small class to 50.0000% of the training data, 8 epochs
Train on 483334 samples, validate on 85296 samples
Epoch 1/8
2s - loss: 3.0886 - val_loss: 0.5368
Epoch 2/8
1s - loss: 0.5087 - val_loss: 0.5420
Epoch 3/8
2s - loss: 0.5069 - val_loss: 0.5095
Epoch 4/8
2s - loss: 0.5043 - val_loss: 0.6866
Epoch 5/8
1s - loss: 0.5067 - val_loss: 0.5092
Epoch 6/8
2s - loss: 0.4987 - val_loss: 0.5193
Epoch 7/8
2s - loss: 0.4987 - val_loss: 0.4964
Epoch 8/8


1it [00:17, 17.38s/it]

1s - loss: 0.4956 - val_loss: 0.7180
batch size 530844
small_class_sample_size 246529
Oversampling small class to 46.4409% of the training data, 8 epochs
Train on 451216 samples, validate on 79628 samples
Epoch 1/8
1s - loss: 0.5015 - val_loss: 0.4881
Epoch 2/8
1s - loss: 0.4906 - val_loss: 0.4881
Epoch 3/8
1s - loss: 0.4884 - val_loss: 0.4850
Epoch 4/8
1s - loss: 0.4883 - val_loss: 0.4845
Epoch 5/8
2s - loss: 0.4857 - val_loss: 0.4890
Epoch 6/8
2s - loss: 0.4842 - val_loss: 0.4874
Epoch 7/8
1s - loss: 0.4811 - val_loss: 0.4778
Epoch 8/8


2it [00:32, 16.80s/it]

...

1s - loss: 0.2282 - val_loss: 0.2269
batch size 295336
small_class_sample_size 11021
Oversampling small class to 3.7318% of the training data, 8 epochs
Train on 251034 samples, validate on 44302 samples
Epoch 1/8
1s - loss: 0.1743 - val_loss: 0.1757
Epoch 2/8
1s - loss: 0.1741 - val_loss: 0.1754
Epoch 3/8
1s - loss: 0.1741 - val_loss: 0.1745
Epoch 4/8
1s - loss: 0.1740 - val_loss: 0.1743
Epoch 5/8
1s - loss: 0.1740 - val_loss: 0.1752
Epoch 6/8
1s - loss: 0.1737 - val_loss: 0.1756
Epoch 7/8
0s - loss: 0.1742 - val_loss: 0.1785
Epoch 8/8


14it [02:44,  9.38s/it]

0s - loss: 0.1742 - val_loss: 0.1745
batch size 284807
small_class_sample_size 492
Oversampling small class to 0.1727% of the training data, 8 epochs
Train on 242085 samples, validate on 42722 samples
Epoch 1/8
1s - loss: 0.0373 - val_loss: 0.0314
Epoch 2/8
0s - loss: 0.0326 - val_loss: 0.0304
Epoch 3/8
0s - loss: 0.0318 - val_loss: 0.0297
Epoch 4/8
1s - loss: 0.0312 - val_loss: 0.0294
Epoch 5/8
0s - loss: 0.0309 - val_loss: 0.0291
Epoch 6/8
0s - loss: 0.0307 - val_loss: 0.0289
Epoch 7/8
1s - loss: 0.0305 - val_loss: 0.0287
Epoch 8/8


15it [02:52,  8.98s/it]

1s - loss: 0.0303 - val_loss: 0.0286


In [16]:
X           = df.drop(['Class'], axis = 1).as_matrix()
y           = df['Class'].as_matrix()
predictions = model.predict_classes(X, batch_size = 2048, verbose = 0)
good_preds  = (predictions == y).sum()
total_preds = X.shape[0]
print('accuracy: %.4f' % (good_preds / total_preds))
print(Counter(predictions))
confusion_matrix(y, predictions)

accuracy: 0.9973
Counter({0: 284471, 1: 336})


array([[284007,    308],
       [   464,     28]])